# Debug Runt Transform
template for testing/debugging runt transforms

In [195]:
import sys
import os
from os import sep
from os.path import dirname, realpath
from pathlib import Path
import logging
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)

def get_cwd(fname, subdir, crunch_dir=realpath(Path.home()) +sep +'crunch' +sep):
    """
    Convenience function to make a directory string for the current file based on inputs.
    Jupyter Notebook in Anaconda invokes the Python interpreter in Anaconda's subdirectory
    which is why changing sys.argv[0] is necessary. In the future a better way to do this
    should be preferred..
    """
    return crunch_dir +subdir +fname
    
def fix_path(cwd):
    """
    Convenience function to fix argv and python path so that jupyter notebook can run the same as
    any script in crunch.
    """
    sys.argv[0] = cwd
    module_path = os.path.abspath(os.path.join('..'))
    if module_path not in sys.path:
        sys.path.append(module_path)

fix_path(get_cwd('runt_debug_template.ipynb', 'mutate' +sep))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from numba import jit, vectorize
from dask import delayed, dataframe as dd

from ipywidgets import interact, interactive, fixed
from IPython.display import display

pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 50)

from common_util import MUTATE_DIR, DT_HOURLY_FREQ, DT_CAL_DAILY_FREQ, reindex_on_time_mask, df_rows_in_year
from common_util import load_json, dump_json, remove_dups_list, list_get_dict, is_empty_df, search_df, benchmark
from data.data_api import DataAPI
from mutate.common import TRANSFORMS_DIR, get_graphs, get_transforms
from mutate.runt import process_transform

## General

### Init DataAPI, Set Logging Level, and Load Transforms

In [3]:
DataAPI.__init__()
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
transforms = get_transforms()

## Transform Debug

### Set Transform
Fill it in (default is ddiff)

In [32]:
trf_name = 'hohlca'
step = transforms[trf_name]
display(step)

{'meta': {'rec_fmt': 'hohlca',
  'var_fmt': 'list',
  'res_freq': 'H',
  'info': 'hourly open, high, low, close, and average aggregation'},
 'fn': {'ser_fn': ['sr', 'sr', 'sr', 'sr', 'stat'],
  'df_fn': 'guax',
  'col_fn': 'ohlca',
  'freq': 'H'},
 'axe': {'rm': ['hrm', 'hrm'],
  'src': ['root', 'root_split_ohlca'],
  'dst': ['hohlca', 'hohlca']},
 'var': [{'val': [True], 'flt': [0]},
  {'val': [True], 'flt': ['h']},
  {'val': [True], 'flt': ['l']},
  {'val': [True], 'flt': [-1]},
  {'stat_type': ['avg']}]}

In [33]:
display(step)

{'meta': {'rec_fmt': 'hohlca',
  'var_fmt': 'list',
  'res_freq': 'H',
  'info': 'hourly open, high, low, close, and average aggregation'},
 'fn': {'ser_fn': ['sr', 'sr', 'sr', 'sr', 'stat'],
  'df_fn': 'guax',
  'col_fn': 'ohlca',
  'freq': 'H'},
 'axe': {'rm': ['hrm', 'hrm'],
  'src': ['root', 'root_split_ohlca'],
  'dst': ['hohlca', 'hohlca']},
 'var': [{'val': [True], 'flt': [0]},
  {'val': [True], 'flt': ['h']},
  {'val': [True], 'flt': ['l']},
  {'val': [True], 'flt': [-1]},
  {'stat_type': ['avg']}]}

In [128]:
hrm_rcs, hrm_dfs = DataAPI.axe_load(step['axe']['rm'])
src_rcs, src_dfs = DataAPI.axe_load(step['axe']['src'])

In [199]:
for datatype in ['pba', 'vol']:
    for asset_name in ['sp_500', 'russell_2000', 'nasdaq_100', 'dow_jones']:
        print(datatype, asset_name)
        src = df_rows_in_year(src_dfs[[asset_name, 'root', 'root_split_ohlca', 'join', datatype]].dropna())
        hrm = df_rows_in_year(hrm_dfs[[asset_name, 'hrm', 'hrm', datatype, datatype]])
        #local = reindex_on_time_mask(src, hrm)
        daysize = src.groupby(pd.Grouper(freq='D')).size()
        incdays = daysize[daysize<8]
        halfdays = incdays[incdays>0]
        display(len(halfdays))
        display(halfdays)

pba sp_500


18

id
2009-11-27 00:00:00+00:00    6
2009-12-24 00:00:00+00:00    5
2010-11-26 00:00:00+00:00    5
2011-11-25 00:00:00+00:00    5
2012-07-03 00:00:00+00:00    5
2012-11-23 00:00:00+00:00    5
2012-12-24 00:00:00+00:00    5
2013-07-03 00:00:00+00:00    5
2013-11-29 00:00:00+00:00    5
2013-12-24 00:00:00+00:00    5
2014-07-03 00:00:00+00:00    5
2014-11-28 00:00:00+00:00    5
2014-12-24 00:00:00+00:00    5
2015-11-27 00:00:00+00:00    5
2015-12-24 00:00:00+00:00    5
2016-11-25 00:00:00+00:00    5
2017-07-03 00:00:00+00:00    5
2017-11-24 00:00:00+00:00    5
dtype: int64

pba russell_2000


24

...

23

id
2009-11-27 00:00:00+00:00    6
2009-12-24 00:00:00+00:00    5
2010-09-24 00:00:00+00:00    7
2010-11-26 00:00:00+00:00    5
2011-11-25 00:00:00+00:00    5
2012-07-03 00:00:00+00:00    5
2012-11-23 00:00:00+00:00    5
2012-12-24 00:00:00+00:00    5
2013-04-25 00:00:00+00:00    6
2013-07-03 00:00:00+00:00    5
2013-11-29 00:00:00+00:00    5
2013-12-24 00:00:00+00:00    5
2014-07-03 00:00:00+00:00    5
2014-11-19 00:00:00+00:00    7
2014-11-28 00:00:00+00:00    5
2014-12-24 00:00:00+00:00    5
2015-11-27 00:00:00+00:00    5
2015-12-24 00:00:00+00:00    5
2016-11-25 00:00:00+00:00    5
2017-01-25 00:00:00+00:00    7
2017-06-29 00:00:00+00:00    6
2017-07-03 00:00:00+00:00    5
2017-11-24 00:00:00+00:00    5
dtype: int64

vol dow_jones


68

id
2009-11-27 00:00:00+00:00    6
2009-12-24 00:00:00+00:00    5
2010-11-26 00:00:00+00:00    5
2011-03-09 00:00:00+00:00    7
2011-11-25 00:00:00+00:00    5
2012-02-06 00:00:00+00:00    7
2012-07-03 00:00:00+00:00    5
2012-11-23 00:00:00+00:00    5
2012-12-24 00:00:00+00:00    5
2013-04-25 00:00:00+00:00    7
2013-07-03 00:00:00+00:00    5
2013-11-29 00:00:00+00:00    5
2013-12-24 00:00:00+00:00    5
2014-07-03 00:00:00+00:00    5
2014-11-28 00:00:00+00:00    5
2014-12-24 00:00:00+00:00    5
2015-11-27 00:00:00+00:00    5
2015-12-24 00:00:00+00:00    5
2016-03-11 00:00:00+00:00    1
2016-04-04 00:00:00+00:00    5
2016-04-05 00:00:00+00:00    6
2016-04-06 00:00:00+00:00    5
2016-04-08 00:00:00+00:00    7
2016-05-10 00:00:00+00:00    1
2016-05-11 00:00:00+00:00    4
2016-05-13 00:00:00+00:00    5
2016-06-02 00:00:00+00:00    7
2016-06-06 00:00:00+00:00    1
2016-06-07 00:00:00+00:00    1
2016-06-08 00:00:00+00:00    1
2016-06-09 00:00:00+00:00    1
2016-06-10 00:00:00+00:00    1
2016-

In [169]:
for day in days:
    dayl = ('{} 00:00'.format(day), '{} 23:00'.format(day))
    display(dayl)
    display(src.dropna().loc[dayl[0]:dayl[1]])
    #display(runt_dfs[asset_keys[-1]].loc[dayl[0]:dayl[1]])

('2009-06-01 00:00', '2009-06-01 23:00')

,pba_open,pba_high,pba_low,pba_close,pba_avgPrice
id,,,,,
2009-06-01 13:00:00+00:00,923.26,938.22,923.26,936.37,933.7776
2009-06-01 14:00:00+00:00,936.75,942.69,936.75,942.13,941.0939
2009-06-01 15:00:00+00:00,942.22,944.14,941.60,943.72,942.4451
2009-06-01 16:00:00+00:00,943.74,945.53,942.82,942.91,944.0086
2009-06-01 17:00:00+00:00,942.77,947.15,942.16,946.89,943.6656
2009-06-01 18:00:00+00:00,947.01,947.77,943.08,944.15,945.2102
2009-06-01 19:00:00+00:00,944.05,946.87,940.03,943.99,943.5008
2009-06-01 20:00:00+00:00,943.91,943.91,942.86,942.87,942.8939


('2010-04-24 00:00', '2010-04-24 23:00')

,pba_open,pba_high,pba_low,pba_close,pba_avgPrice
id,,,,,


('2011-01-28 00:00', '2011-01-28 23:00')

...

,pba_open,pba_high,pba_low,pba_close,pba_avgPrice
id,,,,,
2015-09-02 13:00:00+00:00,1916.52,1938.37,1916.52,1930.84,1933.8087
2015-09-02 14:00:00+00:00,1930.88,1935.71,1919.64,1924.89,1926.9341
2015-09-02 15:00:00+00:00,1924.91,1931.59,1920.45,1923.10,1926.2600
2015-09-02 16:00:00+00:00,1923.10,1934.99,1920.29,1934.46,1926.1309
2015-09-02 17:00:00+00:00,1934.45,1938.03,1931.68,1933.66,1934.6466
2015-09-02 18:00:00+00:00,1933.66,1939.68,1929.32,1935.24,1935.2915
2015-09-02 19:00:00+00:00,1935.26,1948.68,1929.00,1948.68,1936.8953
2015-09-02 20:00:00+00:00,1948.64,1948.91,1948.64,1948.86,1948.8598


('2016-03-28 00:00', '2016-03-28 23:00')

,pba_open,pba_high,pba_low,pba_close,pba_avgPrice
id,,,,,
2016-03-28 13:00:00+00:00,2037.89,2040.74,2035.29,2036.26,2037.0913
2016-03-28 14:00:00+00:00,2036.28,2038.65,2031.96,2034.59,2035.8234
2016-03-28 15:00:00+00:00,2034.60,2037.36,2032.72,2037.11,2035.3500
2016-03-28 16:00:00+00:00,2037.11,2040.94,2035.26,2040.79,2038.0200
2016-03-28 17:00:00+00:00,2040.79,2040.91,2038.73,2040.26,2039.7654
2016-03-28 18:00:00+00:00,2040.26,2042.67,2035.93,2036.87,2039.7315
2016-03-28 19:00:00+00:00,2036.89,2040.77,2035.46,2037.09,2038.5399
2016-03-28 20:00:00+00:00,2037.09,2037.09,2037.04,2037.05,2037.0518


('2017-04-21 00:00', '2017-04-21 23:00')

,pba_open,pba_high,pba_low,pba_close,pba_avgPrice
id,,,,,
2017-04-21 13:00:00+00:00,2354.74,2356.18,2352.42,2353.53,2354.4362
2017-04-21 14:00:00+00:00,2353.58,2355.17,2350.58,2351.79,2353.4016
2017-04-21 15:00:00+00:00,2351.79,2354.13,2348.85,2353.25,2351.9150
2017-04-21 16:00:00+00:00,2353.25,2353.27,2349.60,2350.27,2351.1380
2017-04-21 17:00:00+00:00,2350.26,2351.53,2344.51,2351.15,2347.4511
2017-04-21 18:00:00+00:00,2351.13,2353.02,2349.90,2351.33,2351.3533
2017-04-21 19:00:00+00:00,2351.34,2352.13,2347.99,2348.90,2350.0469
2017-04-21 20:00:00+00:00,2348.98,2348.98,2348.69,2348.69,2348.7114


In [170]:
for day in days:
    dayl = ('{} 00:00'.format(day), '{} 23:00'.format(day))
    display(dayl)
    display(local.loc[dayl[0]:dayl[1]])
    #display(runt_dfs[asset_keys[-1]].loc[dayl[0]:dayl[1]])

('2009-06-01 00:00', '2009-06-01 23:00')

,pba_open,pba_high,pba_low,pba_close,pba_avgPrice
id,,,,,
2009-06-01 08:00:00+00:00,923.26,938.22,923.26,936.37,933.7776
2009-06-01 09:00:00+00:00,936.75,942.69,936.75,942.13,941.0939
2009-06-01 10:00:00+00:00,942.22,944.14,941.60,943.72,942.4451
2009-06-01 11:00:00+00:00,943.74,945.53,942.82,942.91,944.0086
2009-06-01 12:00:00+00:00,942.77,947.15,942.16,946.89,943.6656
2009-06-01 13:00:00+00:00,947.01,947.77,943.08,944.15,945.2102
2009-06-01 14:00:00+00:00,944.05,946.87,940.03,943.99,943.5008
2009-06-01 15:00:00+00:00,943.91,943.91,942.86,942.87,942.8939


('2010-04-24 00:00', '2010-04-24 23:00')

,pba_open,pba_high,pba_low,pba_close,pba_avgPrice
id,,,,,


('2011-01-28 00:00', '2011-01-28 23:00')

...

,pba_open,pba_high,pba_low,pba_close,pba_avgPrice
id,,,,,
2015-09-02 08:00:00+00:00,1916.52,1938.37,1916.52,1930.84,1933.8087
2015-09-02 09:00:00+00:00,1930.88,1935.71,1919.64,1924.89,1926.9341
2015-09-02 10:00:00+00:00,1924.91,1931.59,1920.45,1923.10,1926.2600
2015-09-02 11:00:00+00:00,1923.10,1934.99,1920.29,1934.46,1926.1309
2015-09-02 12:00:00+00:00,1934.45,1938.03,1931.68,1933.66,1934.6466
2015-09-02 13:00:00+00:00,1933.66,1939.68,1929.32,1935.24,1935.2915
2015-09-02 14:00:00+00:00,1935.26,1948.68,1929.00,1948.68,1936.8953
2015-09-02 15:00:00+00:00,1948.64,1948.91,1948.64,1948.86,1948.8598


('2016-03-28 00:00', '2016-03-28 23:00')

,pba_open,pba_high,pba_low,pba_close,pba_avgPrice
id,,,,,
2016-03-28 08:00:00+00:00,2037.89,2040.74,2035.29,2036.26,2037.0913
2016-03-28 09:00:00+00:00,2036.28,2038.65,2031.96,2034.59,2035.8234
2016-03-28 10:00:00+00:00,2034.60,2037.36,2032.72,2037.11,2035.3500
2016-03-28 11:00:00+00:00,2037.11,2040.94,2035.26,2040.79,2038.0200
2016-03-28 12:00:00+00:00,2040.79,2040.91,2038.73,2040.26,2039.7654
2016-03-28 13:00:00+00:00,2040.26,2042.67,2035.93,2036.87,2039.7315
2016-03-28 14:00:00+00:00,2036.89,2040.77,2035.46,2037.09,2038.5399
2016-03-28 15:00:00+00:00,2037.09,2037.09,2037.04,2037.05,2037.0518


('2017-04-21 00:00', '2017-04-21 23:00')

,pba_open,pba_high,pba_low,pba_close,pba_avgPrice
id,,,,,
2017-04-21 08:00:00+00:00,2354.74,2356.18,2352.42,2353.53,2354.4362
2017-04-21 09:00:00+00:00,2353.58,2355.17,2350.58,2351.79,2353.4016
2017-04-21 10:00:00+00:00,2351.79,2354.13,2348.85,2353.25,2351.9150
2017-04-21 11:00:00+00:00,2353.25,2353.27,2349.60,2350.27,2351.1380
2017-04-21 12:00:00+00:00,2350.26,2351.53,2344.51,2351.15,2347.4511
2017-04-21 13:00:00+00:00,2351.13,2353.02,2349.90,2351.33,2351.3533
2017-04-21 14:00:00+00:00,2351.34,2352.13,2347.99,2348.90,2350.0469
2017-04-21 15:00:00+00:00,2348.98,2348.98,2348.69,2348.69,2348.7114


### Test Section

In [34]:
runt_rcs, runt_dfs = process_transform(step, dump=False)

/home/kev/miniconda3/lib/python3.7/site-packages/pyarrow/pandas_compat.py:708: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels = getattr(columns, 'labels', None) or [
/home/kev/miniconda3/lib/python3.7/site-packages/pyarrow/pandas_compat.py:735: FutureWarning: the 'labels' keyword is deprecated, use 'codes' instead
  return pd.MultiIndex(levels=new_levels, labels=labels, names=columns.names)
/home/kev/miniconda3/lib/python3.7/site-packages/pyarrow/pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


INFO:root:data: ['dow_jones', 'root', 'root_split_ohlca', 'join', 'pba']
DEBUG:root:rm_match: [['dow_jones', 'hrm', 'hrm', 'pba', 'pba']]
DEBUG:root:pre-runt:                            pba_open  pba_high   pba_low  pba_close  \
id                                                                   
1996-01-02 14:00:00+00:00   5115.67   5115.67   5098.69    5111.70   
1996-01-02 15:00:00+00:00   5112.42   5159.75   5112.42    5157.22   
1996-01-02 16:00:00+00:00   5157.22   5157.22   5144.22    5151.80   
1996-01-02 17:00:00+00:00   5151.44   5159.39   5148.55    5158.67   
1996-01-02 18:00:00+00:00   5158.67   5165.53   5156.50    5163.00   
...                             ...       ...       ...        ...   
2018-01-08 17:00:00+00:00  25259.27  25276.45  25253.96   25275.32   
2018-01-08 18:00:00+00:00  25275.07  25311.99  25275.07   25299.05   
2018-01-08 19:00:00+00:00  25298.96  25305.34  25289.86   25299.16   
2018-01-08 20:00:00+00:00  25299.39  25301.22  25265.93   25278.24   
2

/home/kev/miniconda3/lib/python3.7/site-packages/pandas/core/generic.py:456: FutureWarning: The default of the 'keep_tz' keyword in DatetimeIndex.to_series will change to True in a future release. You can set 'keep_tz=True' to obtain the future behaviour and silence this warning.
  s = level_values.to_series()
/home/kev/miniconda3/lib/python3.7/site-packages/pandas/core/generic.py:464: FutureWarning: The default of the 'keep_tz' keyword in DatetimeIndex.to_series will change to True in a future release. You can set 'keep_tz=True' to obtain the future behaviour and silence this warning.
  dindex = axis_index.to_series()


DEBUG:root:mutate_type: hohlca
DEBUG:root:post-runt:                            pba_open  pba_high   pba_low  pba_close  \
id                                                                   
2009-01-02 14:00:00+00:00   8771.77   8817.65   8761.10    8783.08   
2009-01-02 15:00:00+00:00   8783.24   8907.97   8768.99    8905.82   
2009-01-02 16:00:00+00:00   8905.98   8934.65   8889.89    8918.17   
2009-01-02 17:00:00+00:00   8918.41   8951.54   8913.87    8931.07   
2009-01-02 18:00:00+00:00   8931.47   8981.57   8925.97    8968.66   
...                             ...       ...       ...        ...   
2017-12-29 17:00:00+00:00  24826.01  24828.58  24810.17   24815.47   
2017-12-29 18:00:00+00:00  24815.35  24817.70  24789.76   24798.39   
2017-12-29 19:00:00+00:00  24798.38  24808.67  24795.19   24805.51   
2017-12-29 20:00:00+00:00  24805.31  24822.22  24735.60   24743.73   
2017-12-29 21:00:00+00:00  24744.45  24749.99  24719.22   24719.22   

                           pba_avgPr

INFO:root:data: ['dow_jones', 'root', 'root_split_ohlca', 'join', 'trmi3_mkt3_news_bondBuzz']
DEBUG:root:rm_match: [['dow_jones', 'hrm', 'hrm', 'trmi3', 'trmi3']]
DEBUG:root:rm_df.index - data_df.index: DatetimeIndex(['1996-01-02 14:00:00+00:00', '1996-01-02 15:00:00+00:00',
               '1996-01-02 16:00:00+00:00', '1996-01-02 17:00:00+00:00',
               '1996-01-02 18:00:00+00:00', '1996-01-02 19:00:00+00:00',
               '1996-01-02 20:00:00+00:00', '1996-01-02 21:00:00+00:00',
               '1996-01-03 14:00:00+00:00', '1996-01-03 15:00:00+00:00',
               ...
               '1997-12-30 20:00:00+00:00', '1997-12-30 21:00:00+00:00',
               '1997-12-31 14:00:00+00:00', '1997-12-31 15:00:00+00:00',
               '1997-12-31 16:00:00+00:00', '1997-12-31 17:00:00+00:00',
               '1997-12-31 18:00:00+00:00', '1997-12-31 19:00:00+00:00',
               '1997-12-31 20:00:00+00:00', '1997-12-31 21:00:00+00:00'],
              dtype='datetime64[ns, UTC]', name

...

DEBUG:root:variant: ({'val': True, 'flt': 0}, {'val': True, 'flt': 'h'}, {'val': True, 'flt': 'l'}, {'val': True, 'flt': -1}, {'stat_type': 'avg'})
DEBUG:root:subvar: {'val': True, 'flt': 0}
DEBUG:root:subvar: {'val': True, 'flt': 'h'}
DEBUG:root:subvar: {'val': True, 'flt': 'l'}
DEBUG:root:subvar: {'val': True, 'flt': -1}
DEBUG:root:subvar: {'stat_type': 'avg'}
DEBUG:root:mutate_type: hohlca
DEBUG:root:post-runt:                            etf3_buzz_N_open  etf3_buzz_N_high  \
id                                                              
2009-01-02 14:00:00+00:00      36828.700000      36828.700000   
2009-01-02 15:00:00+00:00      38001.300000      38001.300000   
2009-01-02 16:00:00+00:00      40739.300000      40739.300000   
2009-01-02 17:00:00+00:00      43636.000000      43636.000000   
2009-01-02 18:00:00+00:00      48227.000000      48227.000000   
...                                     ...               ...   
2017-12-29 17:00:00+00:00     127034.600007     127034.600007 

DEBUG:root:variant: ({'val': True, 'flt': 0}, {'val': True, 'flt': 'h'}, {'val': True, 'flt': 'l'}, {'val': True, 'flt': -1}, {'stat_type': 'avg'})
DEBUG:root:subvar: {'val': True, 'flt': 0}
DEBUG:root:subvar: {'val': True, 'flt': 'h'}
DEBUG:root:subvar: {'val': True, 'flt': 'l'}
DEBUG:root:subvar: {'val': True, 'flt': -1}
DEBUG:root:subvar: {'stat_type': 'avg'}
DEBUG:root:mutate_type: hohlca
DEBUG:root:post-runt:                            mkt3_bondBuzz_N_open  mkt3_bondBuzz_N_high  \
id                                                                      
2009-01-02 14:00:00+00:00                1643.9                1643.9   
2009-01-02 15:00:00+00:00                1915.3                1915.3   
2009-01-02 16:00:00+00:00                2245.2                2245.2   
2009-01-02 17:00:00+00:00                2481.6                2481.6   
2009-01-02 18:00:00+00:00                2797.6                2797.6   
...                                         ...                   ...  

DEBUG:root:pre-runt:                            mkt3_stockIndexBuzz_N
id                                              
1998-01-02 14:00:00+00:00              62.000000
1998-01-02 15:00:00+00:00              62.000000
1998-01-02 16:00:00+00:00              62.000000
1998-01-02 17:00:00+00:00              62.000000
1998-01-02 18:00:00+00:00             398.500000
...                                          ...
2018-01-08 17:00:00+00:00            7813.999999
2018-01-08 18:00:00+00:00            8511.199999
2018-01-08 19:00:00+00:00            9083.199999
2018-01-08 20:00:00+00:00            9465.199999
2018-01-08 21:00:00+00:00            9830.300000

[39888 rows x 1 columns]
DEBUG:root:variant: ({'val': True, 'flt': 0}, {'val': True, 'flt': 'h'}, {'val': True, 'flt': 'l'}, {'val': True, 'flt': -1}, {'stat_type': 'avg'})
DEBUG:root:subvar: {'val': True, 'flt': 0}
DEBUG:root:subvar: {'val': True, 'flt': 'h'}
DEBUG:root:subvar: {'val': True, 'flt': 'l'}
DEBUG:root:subvar: {'val': True, 'f

INFO:root:data: ['sp_500', 'root', 'root_split_ohlca', 'join', 'trmi3_mkt3_social_ratesBuzz']
DEBUG:root:rm_match: [['sp_500', 'hrm', 'hrm', 'trmi3', 'trmi3']]
DEBUG:root:rm_df.index - data_df.index: DatetimeIndex(['1996-01-02 14:00:00+00:00', '1996-01-02 15:00:00+00:00',
               '1996-01-02 16:00:00+00:00', '1996-01-02 17:00:00+00:00',
               '1996-01-02 18:00:00+00:00', '1996-01-02 19:00:00+00:00',
               '1996-01-02 20:00:00+00:00', '1996-01-02 21:00:00+00:00',
               '1996-01-03 14:00:00+00:00', '1996-01-03 15:00:00+00:00',
               ...
               '1997-12-30 20:00:00+00:00', '1997-12-30 21:00:00+00:00',
               '1997-12-31 14:00:00+00:00', '1997-12-31 15:00:00+00:00',
               '1997-12-31 16:00:00+00:00', '1997-12-31 17:00:00+00:00',
               '1997-12-31 18:00:00+00:00', '1997-12-31 19:00:00+00:00',
               '1997-12-31 20:00:00+00:00', '1997-12-31 21:00:00+00:00'],
              dtype='datetime64[ns, UTC]', name='i

INFO:root:data: ['sp_500', 'root', 'root_split_ohlca', 'join', 'vol']
DEBUG:root:rm_match: [['sp_500', 'hrm', 'hrm', 'vol', 'vol']]
DEBUG:root:pre-runt:                            vol_open  vol_high  vol_low  vol_close  \
id                                                                  
1996-01-02 15:00:00+00:00     14.46     14.46    13.30      13.32   
1996-01-02 16:00:00+00:00     13.49     13.61    13.34      13.50   
1996-01-02 17:00:00+00:00     13.47     13.49    13.20      13.31   
1996-01-02 18:00:00+00:00     13.29     13.56    13.29      13.36   
1996-01-02 19:00:00+00:00     13.47     13.52    13.18      13.35   
...                             ...       ...      ...        ...   
2018-01-08 17:00:00+00:00      9.59      9.60     9.52       9.54   
2018-01-08 18:00:00+00:00      9.53      9.59     9.46       9.47   
2018-01-08 19:00:00+00:00      9.48      9.48     9.34       9.34   
2018-01-08 20:00:00+00:00      9.34      9.41     9.32       9.41   
2018-01-08 21:00:00

In [119]:
asset_name = 'nasdaq_100'
days = ['2009-06-01', '2010-04-24', '2011-01-28', '2011-11-08', '2012-10-09', '2012-11-01', '2014-07-03', '2015-09-02', '2016-03-28', '2017-04-21']

In [120]:
asset_keys = [key for key in runt_dfs.keys() if key[0]==asset_name]

In [121]:
for day in days:
    dayl = ('{} 00:00'.format(day), '{} 23:00'.format(day))
    display(dayl)
    display(runt_dfs[asset_keys[0]].loc[dayl[0]:dayl[1]])
    #display(runt_dfs[asset_keys[-1]].loc[dayl[0]:dayl[1]])

('2009-06-01 00:00', '2009-06-01 23:00')

,pba_open,pba_high,pba_low,pba_close,pba_avgPrice
id,,,,,
2009-06-01 16:00:00+00:00,1475.05,1480.05,1473.70,1475.48,1476.6256
2009-06-01 17:00:00+00:00,1475.23,1481.19,1472.53,1480.79,1475.3252
2009-06-01 18:00:00+00:00,1480.90,1482.18,1474.94,1478.30,1478.5643
2009-06-01 19:00:00+00:00,1478.29,1479.62,1470.34,1477.54,1475.8310


('2010-04-24 00:00', '2010-04-24 23:00')

,pba_open,pba_high,pba_low,pba_close,pba_avgPrice
id,,,,,


('2011-01-28 00:00', '2011-01-28 23:00')

...

,pba_open,pba_high,pba_low,pba_close,pba_avgPrice
id,,,,,
2015-09-02 13:00:00+00:00,4207.682,4209.073,4184.064,4190.781,4195.7504
2015-09-02 14:00:00+00:00,4190.762,4207.500,4166.080,4183.870,4184.8931
2015-09-02 15:00:00+00:00,4183.860,4200.773,4177.646,4184.489,4188.4469
2015-09-02 16:00:00+00:00,4184.489,4210.343,4173.437,4209.078,4187.0637
2015-09-02 17:00:00+00:00,4209.078,4220.099,4202.030,4209.339,4212.0655
2015-09-02 18:00:00+00:00,4209.339,4227.984,4196.486,4214.457,4213.9425
2015-09-02 19:00:00+00:00,4214.457,4255.103,4200.270,4255.103,4221.2942
2015-09-02 20:00:00+00:00,4256.176,4256.230,4256.176,4256.230,4256.2300


('2016-03-28 00:00', '2016-03-28 23:00')

,pba_open,pba_high,pba_low,pba_close,pba_avgPrice
id,,,,,
2016-03-28 13:00:00+00:00,4415.044,4417.570,4393.077,4399.030,4400.8347
2016-03-28 14:00:00+00:00,4399.069,4413.255,4396.430,4402.058,4404.4811
2016-03-28 15:00:00+00:00,4402.042,4409.433,4397.748,4408.508,4404.1108
2016-03-28 16:00:00+00:00,4408.443,4412.063,4401.787,4410.123,4406.9870
2016-03-28 17:00:00+00:00,4410.109,4410.141,4401.277,4406.722,4404.7084
2016-03-28 18:00:00+00:00,4406.729,4413.675,4394.217,4396.848,4405.1043
2016-03-28 19:00:00+00:00,4396.952,4406.860,4394.728,4398.348,4401.8767
2016-03-28 20:00:00+00:00,4398.348,4398.348,4398.066,4398.066,4398.0660


('2017-04-21 00:00', '2017-04-21 23:00')

,pba_open,pba_high,pba_low,pba_close,pba_avgPrice
id,,,,,
2017-04-21 13:00:00+00:00,5447.275,5447.488,5435.238,5439.440,5441.1371
2017-04-21 14:00:00+00:00,5439.440,5445.856,5437.497,5439.856,5442.1327
2017-04-21 15:00:00+00:00,5439.903,5450.812,5436.617,5449.199,5444.6348
2017-04-21 16:00:00+00:00,5449.190,5449.521,5441.032,5441.914,5444.7938
2017-04-21 17:00:00+00:00,5441.907,5442.971,5430.224,5441.461,5435.6335
2017-04-21 18:00:00+00:00,5441.395,5441.887,5436.534,5439.713,5439.8235
2017-04-21 19:00:00+00:00,5439.723,5444.754,5433.644,5442.403,5438.9384
2017-04-21 20:00:00+00:00,5442.096,5442.096,5442.049,5442.049,5442.0490


In [108]:
runt_dfs[asset_keys[0]].groupby(pd.Grouper(freq='D')).size().value_counts()

8    2245
0    1019
5      17
9       2
6       1
dtype: int64